In [1]:
# load data
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('dogImages/train')
valid_files, valid_targets = load_dataset('dogImages/valid')
test_files, test_targets = load_dataset('dogImages/test')

# load list of dog names
dog_names = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]

Using TensorFlow backend.


In [2]:
train_files[:5]

array(['dogImages/train/095.Kuvasz/Kuvasz_06442.jpg',
       'dogImages/train/057.Dalmatian/Dalmatian_04054.jpg',
       'dogImages/train/088.Irish_water_spaniel/Irish_water_spaniel_06014.jpg',
       'dogImages/train/008.American_staffordshire_terrier/American_staffordshire_terrier_00596.jpg',
       'dogImages/train/008.American_staffordshire_terrier/American_staffordshire_terrier_00563.jpg'], 
      dtype='<U99')

In [3]:
#np_utils.to_categorical([0,1,2,3,3], 5)

In [4]:
## Human face detector

In [5]:
# import human dataset
import random
random.seed(4)

# load filenames in shuffled human dataset
human_files = np.array(glob("lfw/*/*"))
random.shuffle(human_files)

# print statistics about the dataset
print('There are %d total human images.' % len(human_files))

There are 13233 total human images.


In [6]:
import cv2                
import matplotlib.pyplot as plt                        
%matplotlib inline                               

# extract pre-trained face detector
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
# returns "True" if face is detected in image stored at img_path
def face_detector(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    return len(faces) > 0

In [7]:
# helper functions
def plot_image(img_path):
    img = cv2.imread(img_path)
    # convert BGR image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    # get bounding box for each detected face
    for (x,y,w,h) in faces:
        # add bounding box to color image
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

    # convert BGR image to RGB for plotting
    cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(cv_rgb)
    plt.show()
    
def plot_image_plain(img_path):
    img = cv2.imread(img_path)
    # convert BGR image to RGB for plotting
    cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(cv_rgb)
    plt.show()

In [8]:
# try using resnet
from keras.applications.resnet50 import ResNet50
# define ResNet50 model
ResNet50_model = ResNet50(weights='imagenet')

In [9]:
# convert and load images
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [25]:
# load and preprocess data
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')
valid_tensors = paths_to_tensor(valid_files).astype('float32')
test_tensors = paths_to_tensor(test_files).astype('float32')

100%|██████████| 836/836 [00:07<00:00, 111.73it/s]


In [11]:
# preprocess image for using with imagenet predictions
def ResNet50_predict_labels(img_path):
    from keras.applications.resnet50 import preprocess_input, decode_predictions
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    return np.argmax(ResNet50_model.predict(img))

In [12]:
### returns "True" if a dog is detected in the image stored at img_path
def dog_detector(img_path):
    prediction = ResNet50_predict_labels(img_path)
    return ((prediction <= 268) & (prediction >= 151)) 

In [27]:
# load convered data back
train_data = np.load(open(bnf_train_name, 'rb'))
train_labels = train_targets[:train_data.shape[0]]
validation_data = np.load(open(bnf_test_name, 'rb'))
validation_labels = valid_targets[:validation_data.shape[0]]
test_data = np.load(open(bnf_valid_name, 'rb'))
test_labels = test_targets[:test_data.shape[0]]

In [28]:
print(train_tensors.shape, train_targets.shape)
print(train_data.shape, train_labels.shape)
print(validation_data.shape, validation_labels.shape)
print(test_data.shape, test_labels.shape)


(6680, 224, 224, 3) (6680, 133)
(6680, 5, 5, 2048) (6680, 133)
(835, 5, 5, 2048) (835, 133)
(836, 5, 5, 2048) (836, 133)


In [77]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import regularizers

model = Sequential()
#model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(GlobalAveragePooling2D(input_shape=train_data.shape[1:]))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(133, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_13  (None, 2048)              0         
_________________________________________________________________
dropout_23 (Dropout)         (None, 2048)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 512)               1049088   
_________________________________________________________________
dropout_24 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 133)               68229     
Total params: 1,117,317.0
Trainable params: 1,117,317.0
Non-trainable params: 0.0
_________________________________________________________________


In [78]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1)
#checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.mymodel.resnet50.hdf5', verbose=1, save_best_only=True)
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.mymodel.inceptionv3.hdf5', verbose=1, save_best_only=True)
model.fit(train_data[:150], train_labels[:150],
          epochs=25,
          batch_size=64,
          validation_data=(validation_data[:50], validation_labels[:50]),
          callbacks=[early_stopping])

Train on 150 samples, validate on 50 samples
Epoch 1/25
150/150 [==============================] - 1s - loss: 5.6757 - acc: 0.0067 - val_loss: 4.7278 - val_acc: 0.0600
Epoch 2/25
150/150 [==============================] - 0s - loss: 4.2157 - acc: 0.1267 - val_loss: 4.5237 - val_acc: 0.1400
Epoch 3/25
150/150 [==============================] - 0s - loss: 3.0877 - acc: 0.3333 - val_loss: 4.3090 - val_acc: 0.2000
Epoch 4/25
150/150 [==============================] - 0s - loss: 2.1835 - acc: 0.5200 - val_loss: 4.0898 - val_acc: 0.3000
Epoch 5/25
150/150 [==============================] - 0s - loss: 1.5300 - acc: 0.6800 - val_loss: 3.9668 - val_acc: 0.3000
Epoch 6/25
150/150 [==============================] - 0s - loss: 1.0697 - acc: 0.7400 - val_loss: 3.8658 - val_acc: 0.3400
Epoch 7/25
150/150 [==============================] - 0s - loss: 0.6570 - acc: 0.8800 - val_loss: 3.7809 - val_acc: 0.3800
Epoch 8/25
150/150 [==============================] - 0s - loss: 0.4060 - acc: 0.9333 - val_lo

In [79]:
### train accuracy
predictions = [np.argmax(model.predict(np.expand_dims(feature, axis=0))) for feature in train_data[:150]]

# report test accuracy
train_accuracy = 100*np.sum(np.array(predictions)==np.argmax(train_labels[:150], axis=1))/len(predictions)
print('Train accuracy: %.4f%%' % train_accuracy)

Train accuracy: 99.3333%


In [80]:
### test accuracy
predictions = [np.argmax(model.predict(np.expand_dims(feature, axis=0))) for feature in test_data[:50]]

# report test accuracy
test_accuracy = 100*np.sum(np.array(predictions)==np.argmax(test_labels[:50], axis=1))/len(predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 44.0000%


In [81]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1)
#checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.mymodel.resnet50.hdf5', verbose=1, save_best_only=True)
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.mymodel.inceptionv3.hdf5', verbose=1, save_best_only=True)
model.fit(train_data, train_labels,
          epochs=25,
          batch_size=64,
          validation_data=(validation_data, validation_labels),
          callbacks=[early_stopping, checkpointer])

Train on 6680 samples, validate on 835 samples
Epoch 1/25
6680/6680 [==============================] - 3s - loss: 1.6465 - acc: 0.6311 - val_loss: 0.6686 - val_acc: 0.8012
Epoch 2/25
6680/6680 [==============================] - 2s - loss: 0.7307 - acc: 0.7825 - val_loss: 0.6116 - val_acc: 0.8204
Epoch 3/25
6680/6680 [==============================] - 2s - loss: 0.5846 - acc: 0.8201 - val_loss: 0.5702 - val_acc: 0.8323
Epoch 4/25
6680/6680 [==============================] - 2s - loss: 0.5107 - acc: 0.8386 - val_loss: 0.5770 - val_acc: 0.8419
Epoch 5/25
6680/6680 [==============================] - 2s - loss: 0.4482 - acc: 0.8573 - val_loss: 0.5747 - val_acc: 0.8180
Epoch 6/25
6680/6680 [==============================] - 3s - loss: 0.3948 - acc: 0.8672 - val_loss: 0.5596 - val_acc: 0.8347
Epoch 7/25
6680/6680 [==============================] - 2s - loss: 0.3505 - acc: 0.8834 - val_loss: 0.6051 - val_acc: 0.8240
Epoch 8/25
6680/6680 [==============================] - 2s - loss: 0.3329 - ac

In [82]:
### train accuracy
predictions = [np.argmax(model.predict(np.expand_dims(feature, axis=0))) for feature in train_data[:150]]

# report test accuracy
train_accuracy = 100*np.sum(np.array(predictions)==np.argmax(train_labels[:150], axis=1))/len(predictions)
print('Train accuracy: %.4f%%' % train_accuracy)

Train accuracy: 96.6667%


In [83]:
### test accuracy
predictions = [np.argmax(model.predict(np.expand_dims(feature, axis=0))) for feature in test_data]

# report test accuracy
test_accuracy = 100*np.sum(np.array(predictions)==np.argmax(test_labels, axis=1))/len(predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 78.4689%


In [84]:
def extract_Resnet50(tensor):
	from keras.applications.resnet50 import ResNet50, preprocess_input
	return ResNet50(weights='imagenet', include_top=False).predict(preprocess_input(tensor))

def extract_InceptionV3(tensor):
	from keras.applications.inception_v3 import InceptionV3, preprocess_input
	return InceptionV3(weights='imagenet', include_top=False).predict(preprocess_input(tensor))

def extract(tensor, model, preprocess_input):
    return model.predict(preprocess_input(tensor))

In [85]:
from keras.applications.inception_v3 import preprocess_input as inception_v3_preprocess_input
def predict_breed(img_path):
    # extract bottleneck features
    #bottleneck_feature = extract_Resnet50(path_to_tensor(img_path))
    bottleneck_feature = extract(path_to_tensor(img_path), inceptionv3_model, inception_v3_preprocess_input)
    # obtain predicted vector
    predicted_vector = model.predict(bottleneck_feature)
    # return dog breed that is predicted by the model
    return dog_names[np.argmax(predicted_vector)]

In [38]:
bottleneck_feature = extract(path_to_tensor("trisha.jpg"), inceptionv3_model, inception_v3_preprocess_input)

In [39]:
bottleneck_feature = path_to_tensor("trisha.jpg")
bottleneck_feature.shape


(1, 224, 224, 3)

In [86]:
#predict_breed("lfw/Jennifer_Renee_Short/Jennifer_Renee_Short_0001.jpg")
predict_breed("trisha.jpg")

'Finnish_spitz'

In [87]:
def detect_breed(img_path):
    is_dog = False
    is_human = False
    if dog_detector(img_path):
        is_dog = True
    elif face_detector(img_path):
        is_human = True
    else:
        print("No dog or humans found")
        plot_image_plain(img_path)
        return
    dog_breed = predict_breed(img_path)
    if(is_human):
        print("Hey there...")
        plot_image_plain(img_path)
        print("You look like a...\n {dog_breed}".format(dog_breed=dog_breed))
    else:
        plot_image_plain(img_path)
        print("This look like a...\n {dog_breed}".format(dog_breed=dog_breed))
        

In [98]:
#detect_breed("lfw/Jennifer_Renee_Short/Jennifer_Renee_Short_0001.jpg")
#detect_breed("dhana.jpg")
import time
t0 = time.time()
for i in range(1, 2):
    detect_breed("nivetha.jpg")
print("time taken", time.time()-t0)
#detect_breed("cat.jpg")
#detect_breed("trisha.jpg")
#detect_breed("sara.jpg")

time taken 0.0002300739288330078
